## Background

## Setup
### Install dependencies

In [1]:
#!pip install tensorflow tensorflow-gpu opencv-python matplotlib

### Import dependencies

In [2]:
import cv2
import numpy as np
import os
import random
from matplotlib import pyplot as plt

In [3]:
# import tensorflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation
import tensorflow as tf

In [4]:
### Set GPU growth
'''
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
'''

'\ngpus = tf.config.experimental.list_physical_devices(\'GPU\')\nfor gpu in gpus:\n    tf.config.experimental.set_memory_growth(gpu, True)\nfor gpu in gpus:\n    print("Name:", gpu.name, "  Type:", gpu.device_type)\n'

### Create folder setup

In [5]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

# Make the directory if it doesn't exist
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

## Collect positives and anchor images
### Untar labelled faces in the wild dataset

In [7]:
!curl http://vis-www.cs.umass.edu/lfw/lfw.tgz -o lfw.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  172M    0 1257k    0     0  1257k      0  0:02:20  0:00:01  0:02:19 1044k
  1  172M    1 3453k    0     0  1726k      0  0:01:42  0:00:02  0:01:40 1566k
  3  172M    3 5851k    0     0  1950k      0  0:01:30  0:00:03  0:01:27 1826k
  4  172M    4 8032k    0     0  2008k      0  0:01:27  0:00:04  0:01:23 1910k
  6  172M    6 10.5M    0     0  2162k      0  0:01:21  0:00:05  0:01:16 2176k
  8  172M    8 13.9M    0     0  2374k      0  0:01:14  0:00:06  0:01:08 2597k
 10  172M   10 17.6M    0     0  2587k      0  0:01:08  0:00:07  0:01:01 2932k
 12  172M   12 21.4M    0     0  2751k      0  0:01:04  0:00:08  0:00:56 3232k
 14  172M   14 25.1M    0     0  2858k      0  0:01

In [39]:
# http://vis-www.cs.umass.edu/lfw/
# Uncompress tar gz labelled faces in the wild dataset
!tar -xvf lfw.tgz

x lfw/
x lfw/George_HW_Bush/
x lfw/George_HW_Bush/George_HW_Bush_0001.jpg
x lfw/George_HW_Bush/George_HW_Bush_0002.jpg
x lfw/George_HW_Bush/George_HW_Bush_0003.jpg
x lfw/George_HW_Bush/George_HW_Bush_0004.jpg
x lfw/George_HW_Bush/George_HW_Bush_0005.jpg
x lfw/George_HW_Bush/George_HW_Bush_0006.jpg
x lfw/George_HW_Bush/George_HW_Bush_0007.jpg
x lfw/George_HW_Bush/George_HW_Bush_0008.jpg
x lfw/George_HW_Bush/George_HW_Bush_0009.jpg
x lfw/George_HW_Bush/George_HW_Bush_0010.jpg
x lfw/George_HW_Bush/George_HW_Bush_0011.jpg
x lfw/George_HW_Bush/George_HW_Bush_0012.jpg
x lfw/George_HW_Bush/George_HW_Bush_0013.jpg
x lfw/Curtis_Strange/
x lfw/Curtis_Strange/Curtis_Strange_0001.jpg
x lfw/Marc_Grossman/
x lfw/Marc_Grossman/Marc_Grossman_0001.jpg
x lfw/Marc_Grossman/Marc_Grossman_0002.jpg
x lfw/Marc_Grossman/Marc_Grossman_0003.jpg
x lfw/Marc_Grossman/Marc_Grossman_0004.jpg
x lfw/Michael_Schumacher/
x lfw/Michael_Schumacher/Michael_Schumacher_0001.jpg
x lfw/Michael_Schumacher/Michael_Schumacher_000

In [40]:
# Move LFW images to the following directory data/negative/
for directroy in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directroy)):
        EX_PATH = os.path.join('lfw', directroy, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0088.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0089.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0090.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0091.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0092.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0093.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0094.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0095.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0096.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0097.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0098.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0099.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0100.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0101.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0102.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0103.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0104.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0105.jpg
x lfw/Gerhard_Schroeder/Gerhard_Schroeder_0106.jpg
x lfw/Gerhard_Schroeder/Gerhard

KeyboardInterrupt: 

### Collect positive and anchor classes

In [57]:
# import uuid library to generate random names for the anchor images
import uuid

# Establish a connection to the webcam
cap =cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Cut down frame to 250x250
    frame = frame[120:120+250,200:200+250,:]
    
    #show image back to screen
    cv2.imshow("Collection", frame)
    
    # Collect anchor image
    if cv2.waitKey(1) & 0xFF == ord('a'):
        # Generate random name
        name = str(uuid.uuid4())
        # Save image
        cv2.imwrite(os.path.join(ANC_PATH, name + '.jpg'), frame)
        print('Anchor image saved')
    
    # Collect positive image
    if cv2.waitKey(1) & 0xFF == ord('p'):
        # Generate random name
        name = str(uuid.uuid4())
        # Save image
        cv2.imwrite(os.path.join(POS_PATH, name + '.jpg'), frame)
        print('Positive image saved')
        
    
    # Break if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam
cap.release()

# Close all windows
cv2.destroyAllWindows()

###############################################################################
# Take almost 300 anchor images and take 300 positive images from each anchor
###############################################################################

Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive image saved
Positive imag

In [ ]:
'''
frame
frame.shape
img = frame[120:120+250,200:200+250,::-1]
plt.imshow(img)
'''

'\nframe\nframe.shape\nimg = frame[120:120+250,200:200+250,::-1]\nplt.imshow(img)\n'

## Load and preprocess images
### Get image paths

In [58]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(300)

In [59]:
# dir_test = anchor.as_numpy_iterator()
# print(next(dir_test))


### Preprocessing - Scale and Normalize

In [60]:
def preprocess(file_path):
    
    # Read the image file
    byte_image = tf.io.read_file(file_path)
    
    # Load the image
    img = tf.image.decode_jpeg(byte_image, channels=3)
    
    # Resize the image to 100x100
    img = tf.image.resize(img, [100, 100])
    
    # Normalize the image
    img = img / 255.0
    return img

In [61]:
# img = preprocess('data\\anchor\\f212266b-622c-40c0-a311-a493b4bef4ea.jpg')
# plt.imshow(img)

### Create labelled dataset

In [62]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(np.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(np.zeros(len(anchor)))))
dataset = positives.concatenate(negatives)

In [69]:
sample = dataset.as_numpy_iterator()
example =sample.next()
example

(b'data\\anchor\\8106f3eb-6924-40fc-9aea-cd61a5fff47a.jpg',
 b'data\\positive\\6649ae21-2546-401b-8a4d-b3205cb84f10.jpg',
 1.0)

### Train test split

In [68]:
def preprocess_twins(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [70]:
preprocess_twins(*example)

(<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
 array([[[0.56985295, 0.60906863, 0.5110294 ],
         [0.564951  , 0.58235294, 0.49779412],
         [0.58259803, 0.5769608 , 0.5210784 ],
         ...,
         [0.3754902 , 0.3529412 , 0.34044117],
         [0.39313725, 0.35196078, 0.32892156],
         [0.39534312, 0.34534314, 0.30710784]],
 
        [[0.5686275 , 0.61519605, 0.5129902 ],
         [0.56666666, 0.5875    , 0.52156866],
         [0.535049  , 0.54093134, 0.4968137 ],
         ...,
         [0.37156862, 0.3507353 , 0.32843137],
         [0.38308823, 0.34509805, 0.30857843],
         [0.4254902 , 0.37083334, 0.33088234]],
 
        [[0.56911767, 0.5894608 , 0.50784314],
         [0.55269605, 0.5602941 , 0.48161766],
         [0.5754902 , 0.57058823, 0.50490195],
         ...,
         [0.34509805, 0.34779412, 0.29705882],
         [0.3747549 , 0.360049  , 0.3007353 ],
         [0.39093137, 0.35882354, 0.29411766]],
 
        ...,
 
        [[0.5769608 , 0.56666666

In [76]:
# Build dataloarder pipeline
data = dataset.map(preprocess_twins)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [77]:
# Train test split
train_data, test_data = data.take(int(len(data)*0.7)), data.skip(int(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)


In [80]:
# train_data
# train_samples = train_data.as_numpy_iterator()
# train_exmp = train_samples.next()
# len(train_exmp[0])

## Model engineering
### Build embedding layer

In [18]:
def make_embedding():
    inp = Input(shape=(100, 100, 3), name='input_image')
    
    # First block
    c1 = Conv2D(64,(10,10),activation='relu', name = 'conv_1')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same', name = 'pool_1' )(c1)
    
    # Second block
    c2 = Conv2D(128,(7,7),activation='relu', name = 'conv_2')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same', name = 'pool_2')(c2)
    
    # Third block
    c3 = Conv2D(128,(4,4),activation='relu', name = 'conv_3')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same', name = 'pool_3')(c3)
    
    # Final embedding block
    c4 = Conv2D(256,(4,4),activation='relu', name = 'conv_4')(m3)
    f1 = Flatten(name = 'flattern')(c4)
    d1 = Dense(4096, activation='relu', name = 'dense')(f1)
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

### Build Distance Layer

In [39]:
class L1Distance(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

### Make Siamese Network

In [57]:
def make_siamese():
    
    # Anchor image in the network
    input_image = Input(name='input_image', shape=(100, 100, 3))
    
    # Validation image in the network
    validation_image = Input(name='validation_image', shape=(100, 100, 3))
    
    # Combine siamese distance component
    embeding = make_embedding()
    L1_distance = L1Distance()
    L1_distance._name = 'l1_distance'
    siamese_distance = L1_distance(embeding(input_image), embeding(validation_image))
    
    # Classification layer
    classifier = Dense(1, activation='sigmoid', name='classifier')(siamese_distance)
    
    return Model(inputs=[input_image, validation_image], outputs=[classifier], name='siamese')

In [58]:
model = make_siamese()
model.summary()

Model: "siamese"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_image (InputLayer)   [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_image[0][0]                
                                                                 validation_image[0][0]           
__________________________________________________________________________________________________
l1_distance (L1Distance)        (None, 4096)         0           embedding[0][0]            